In [1]:
import pandas as pd
import plotly.graph_objects as go
from evento import ajustar_datas

In [2]:
# Fundo sem histórico completo de preços no Yahoo Finance
# Carregar histórico da B3
knri = (
    pd.read_csv('data/knri.csv', parse_dates=['datneg'])
    # a série da B3 começa em "2010-12-01" -> vamos começar em "2011-01-01"
    .query('datneg >= "2011-01-01"')
)
knri

,datneg,preult
20,2011-01-04,100.00
21,2011-01-05,99.70
22,2011-01-06,99.70
23,2011-01-07,99.50
24,2011-01-10,95.50
...,...,...
2993,2023-01-09,139.55
2994,2023-01-10,139.55
2995,2023-01-11,139.97
2996,2023-01-12,139.40


In [3]:
# https://www.ibge.gov.br/estatisticas/economicas/precos-e-custos/9256-indice-nacional-de-precos-ao-consumidor-amplo.html?t=downloads&utm_source=landing&utm_medium=explica&utm_campaign=inflacao#plano-real-mes
ipca = (pd
    .read_csv('data/ipca.csv', parse_dates=['mes'])
    .query('mes >= "2011-01-01"')
    # .rename(columns={'mes':'datneg'})
    .reset_index(drop=True)
)
ipca

,mes,indice
0,2011-01-01,3222.42
1,2011-02-01,3248.20
2,2011-03-01,3273.86
3,2011-04-01,3299.07
4,2011-05-01,3314.58
...,...,...
139,2022-08-01,6388.87
140,2022-09-01,6370.34
141,2022-10-01,6407.93
142,2022-11-01,6434.20


In [4]:
# Normalizar o indice de preços para 1 no início da série
ipca['indice'] = ipca['indice'] / ipca['indice'].iloc[0]
# Criar colunas "ano_mes" como integer para usar como chave de merge
ipca['ano_mes'] = ipca['mes'].dt.year * 100 + ipca['mes'].dt.month
ipca

,mes,indice,ano_mes
0,2011-01-01,1.000000,201101
1,2011-02-01,1.008000,201102
2,2011-03-01,1.015963,201103
3,2011-04-01,1.023786,201104
4,2011-05-01,1.028600,201105
...,...,...,...
139,2022-08-01,1.982631,202208
140,2022-09-01,1.976881,202209
141,2022-10-01,1.988546,202210
142,2022-11-01,1.996698,202211


In [5]:
# Histórico de dividendos do HGLG11 extraído do site Funds Explorer
divs = (
    pd.read_csv('data/knri_divs.csv', parse_dates=['data_com', 'data_pag'])
    # a série da B3 começa em "2010-12-01" -> vamos começar em "2011-01-01"
    .query('data_com >= "2011-01-01"')
)
divs

,tipo,data_com,data_pag,dividendo
1,Rendimento,2011-01-31,2011-02-14,0.675
2,Rendimento,2011-02-28,2011-03-16,0.700
3,Rendimento,2011-03-31,2011-04-14,0.700
4,Rendimento,2011-04-29,2011-05-13,0.700
5,Rendimento,2011-05-31,2011-06-14,0.700
...,...,...,...,...
140,Rendimento,2022-08-31,2022-09-15,0.910
141,Rendimento,2022-09-30,2022-10-17,0.910
142,Rendimento,2022-10-31,2022-11-16,0.910
143,Rendimento,2022-11-30,2022-12-14,0.910


In [6]:
# Criar coluna "ano_mes" como integer para usar como chave de merge
divs['ano_mes'] = divs['data_com'].dt.year * 100 + divs['data_com'].dt.month
divs

,tipo,data_com,data_pag,dividendo,ano_mes
1,Rendimento,2011-01-31,2011-02-14,0.675,201101
2,Rendimento,2011-02-28,2011-03-16,0.700,201102
3,Rendimento,2011-03-31,2011-04-14,0.700,201103
4,Rendimento,2011-04-29,2011-05-13,0.700,201104
5,Rendimento,2011-05-31,2011-06-14,0.700,201105
...,...,...,...,...,...
140,Rendimento,2022-08-31,2022-09-15,0.910,202208
141,Rendimento,2022-09-30,2022-10-17,0.910,202209
142,Rendimento,2022-10-31,2022-11-16,0.910,202210
143,Rendimento,2022-11-30,2022-12-14,0.910,202211


In [7]:
# Adicionar a coluna do indexador de inflação ao histórico de dividendos
divs = divs.merge(ipca, how='inner')
divs

,tipo,data_com,data_pag,dividendo,ano_mes,mes,indice
0,Rendimento,2011-01-31,2011-02-14,0.675,201101,2011-01-01,1.000000
1,Rendimento,2011-02-28,2011-03-16,0.700,201102,2011-02-01,1.008000
2,Rendimento,2011-03-31,2011-04-14,0.700,201103,2011-03-01,1.015963
3,Rendimento,2011-04-29,2011-05-13,0.700,201104,2011-04-01,1.023786
4,Rendimento,2011-05-31,2011-06-14,0.700,201105,2011-05-01,1.028600
...,...,...,...,...,...,...,...
139,Rendimento,2022-08-31,2022-09-15,0.910,202208,2022-08-01,1.982631
140,Rendimento,2022-09-30,2022-10-17,0.910,202209,2022-09-01,1.976881
141,Rendimento,2022-10-31,2022-11-16,0.910,202210,2022-10-01,1.988546
142,Rendimento,2022-11-30,2022-12-14,0.910,202211,2022-11-01,1.996698


In [8]:
# Fazer gráfico de barras com o dividendos pagos pela KNRI11 a cada ano usano o plotly
divs["ano"] = divs["data_com"].dt.year
# Agrupar os dividendos por ano
divs_gb = divs.groupby("ano")["dividendo"].sum().reset_index()
# Criar o gráfico
fig = go.Figure(
    data=[
        go.Bar(
            x=divs_gb["ano"],
            y=divs_gb["dividendo"],
            text=divs_gb["dividendo"].round(1),
            textposition="auto",
        )
    ]
)
fig.update_layout(
    title="Dividendo anual por cota pago pelo KNRI11",
    xaxis_title="Ano",
    yaxis_title="Dividendos (R$)",
    title_x=0.5,
    title_y=0.85,

)
fig.show()

In [9]:
# Houve amortização de capital?
divs["tipo"].value_counts()

Rendimento    144
Name: tipo, dtype: int64

In [10]:
# As colunas "tipo" e "ano" não serão mais usadas
divs.drop(columns=["tipo", "ano"], inplace=True)

In [11]:
# Verificar a quantidade de datas a serem ajustadas
print(len(set(divs['data_com']) - set(ajustar_datas(divs['data_com'], knri['datneg']))), 'datas-com a serem ajustadas')
# Forçar data-com -> datas de negociação do ativo
# anterior=True (padrão) -> se não houver negociação na data-com, ajustar para a primeira data anterior de negociação
divs['data_com'] = ajustar_datas(divs['data_com'], knri['datneg'])
# anterior=False -> se não houver negociação na data de pagamento, ajustar para a primeira data posterior
divs['data_pag'] = ajustar_datas(divs['data_pag'], knri['datneg'], anterior=False)
divs

0 datas-com a serem ajustadas


,data_com,data_pag,dividendo,ano_mes,mes,indice
0,2011-01-31,2011-02-14,0.675,201101,2011-01-01,1.000000
1,2011-02-28,2011-03-16,0.700,201102,2011-02-01,1.008000
2,2011-03-31,2011-04-14,0.700,201103,2011-03-01,1.015963
3,2011-04-29,2011-05-13,0.700,201104,2011-04-01,1.023786
4,2011-05-31,2011-06-14,0.700,201105,2011-05-01,1.028600
...,...,...,...,...,...,...
139,2022-08-31,2022-09-15,0.910,202208,2022-08-01,1.982631
140,2022-09-30,2022-10-17,0.910,202209,2022-09-01,1.976881
141,2022-10-31,2022-11-16,0.910,202210,2022-10-01,1.988546
142,2022-11-30,2022-12-14,0.910,202211,2022-11-01,1.996698


In [12]:
# Verificar se existe alguma "data_com" repetida
divs['data_com'].duplicated().any()

False

In [13]:
# Calcular o dividend yield de 3m, 6m e 12m
# 3m
divs['div_mms3'] = divs['dividendo'].rolling(3).mean()
# 6m
divs['div_mms6'] = divs['dividendo'].rolling(6).mean()
# 12m
divs['div_mms12'] = divs['dividendo'].rolling(12).mean()
# "data_pag" não será usada
divs.drop(columns=['data_pag'], inplace=True)
# Renomear coluna "data_com" para "datneg"
divs.rename(columns={'data_com': 'datneg'}, inplace=True)
divs


,datneg,dividendo,ano_mes,mes,indice,div_mms3,div_mms6,div_mms12
0,2011-01-31,0.675,201101,2011-01-01,1.000000,NaN,NaN,NaN
1,2011-02-28,0.700,201102,2011-02-01,1.008000,NaN,NaN,NaN
2,2011-03-31,0.700,201103,2011-03-01,1.015963,0.691667,NaN,NaN
3,2011-04-29,0.700,201104,2011-04-01,1.023786,0.700000,NaN,NaN
4,2011-05-31,0.700,201105,2011-05-01,1.028600,0.700000,NaN,NaN
...,...,...,...,...,...,...,...,...
139,2022-08-31,0.910,202208,2022-08-01,1.982631,0.886667,0.858333,0.830000
140,2022-09-30,0.910,202209,2022-09-01,1.976881,0.910000,0.875000,0.841667
141,2022-10-31,0.910,202210,2022-10-01,1.988546,0.910000,0.886667,0.853333
142,2022-11-30,0.910,202211,2022-11-01,1.996698,0.910000,0.898333,0.865000


In [14]:
# Calcular o valor real dos dividendos
divs['div_mms6_real'] = divs['div_mms6'] / divs['indice']
divs

,datneg,dividendo,ano_mes,mes,indice,div_mms3,div_mms6,div_mms12,div_mms6_real
0,2011-01-31,0.675,201101,2011-01-01,1.000000,NaN,NaN,NaN,NaN
1,2011-02-28,0.700,201102,2011-02-01,1.008000,NaN,NaN,NaN,NaN
2,2011-03-31,0.700,201103,2011-03-01,1.015963,0.691667,NaN,NaN,NaN
3,2011-04-29,0.700,201104,2011-04-01,1.023786,0.700000,NaN,NaN,NaN
4,2011-05-31,0.700,201105,2011-05-01,1.028600,0.700000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
139,2022-08-31,0.910,202208,2022-08-01,1.982631,0.886667,0.858333,0.830000,0.432926
140,2022-09-30,0.910,202209,2022-09-01,1.976881,0.910000,0.875000,0.841667,0.442616
141,2022-10-31,0.910,202210,2022-10-01,1.988546,0.910000,0.886667,0.853333,0.445887
142,2022-11-30,0.910,202211,2022-11-01,1.996698,0.910000,0.898333,0.865000,0.449909


In [15]:
# Plotar o valor real dos dividendos
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=divs['datneg'],
    y=divs['div_mms6_real'],
    mode='lines+markers',
    name='Dividendos',
))
fig.update_layout(
    title={
        'text':'Dividendos Reais do KNRI11 <br>(média mensal dos últimos 6m)',
        'y':0.85,
        'x':0.5,
    },
    font_family="Open Sans",
    xaxis_title='Data',
    yaxis_title='Valor Real',
)
fig.show()

In [16]:
# Juntar o histórico de preços com o histórico de dividendos
knri = pd.merge(knri, divs, how="outer")
knri

,datneg,preult,dividendo,ano_mes,mes,indice,div_mms3,div_mms6,div_mms12,div_mms6_real
0,2011-01-04,100.00,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
1,2011-01-05,99.70,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
2,2011-01-06,99.70,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
3,2011-01-07,99.50,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
4,2011-01-10,95.50,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2973,2023-01-09,139.55,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
2974,2023-01-10,139.55,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
2975,2023-01-11,139.97,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
2976,2023-01-12,139.40,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN


In [17]:
# Verificar o início do dataframe para ver se está tudo certo
knri.loc[15:20]

,datneg,preult,dividendo,ano_mes,mes,indice,div_mms3,div_mms6,div_mms12,div_mms6_real
15,2011-01-27,97.00,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
16,2011-01-28,96.60,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
17,2011-01-31,96.99,0.675,201101.0,2011-01-01,1.0,NaN,NaN,NaN,NaN
18,2011-02-01,96.99,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
19,2011-02-02,99.00,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
20,2011-02-03,98.50,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN


In [18]:
# Preencher as médias móveis simples para frente com o último valor válido
knri['div_mms3'].fillna(method='ffill', inplace=True)
knri['div_mms6'].fillna(method='ffill', inplace=True)
knri['div_mms12'].fillna(method='ffill', inplace=True)
knri

,datneg,preult,dividendo,ano_mes,mes,indice,div_mms3,div_mms6,div_mms12,div_mms6_real
0,2011-01-04,100.00,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
1,2011-01-05,99.70,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
2,2011-01-06,99.70,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
3,2011-01-07,99.50,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
4,2011-01-10,95.50,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2973,2023-01-09,139.55,NaN,NaN,NaT,NaN,0.94,0.925,0.875,NaN
2974,2023-01-10,139.55,NaN,NaN,NaT,NaN,0.94,0.925,0.875,NaN
2975,2023-01-11,139.97,NaN,NaN,NaT,NaN,0.94,0.925,0.875,NaN
2976,2023-01-12,139.40,NaN,NaN,NaT,NaN,0.94,0.925,0.875,NaN


In [19]:
# Calcular os dividend yields e anualizar o resultado
knri['dy_3m'] = (knri['div_mms3'] / knri['preult'] + 1) ** 12 - 1
knri['dy_6m'] = (knri['div_mms6'] / knri['preult'] + 1) ** 12 - 1
knri['dy_12m'] = (knri['div_mms12'] / knri['preult'] + 1) ** 12 - 1
knri



,datneg,preult,dividendo,ano_mes,mes,indice,div_mms3,div_mms6,div_mms12,div_mms6_real,dy_3m,dy_6m,dy_12m
0,2011-01-04,100.00,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011-01-05,99.70,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011-01-06,99.70,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-01-07,99.50,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-01-10,95.50,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2973,2023-01-09,139.55,NaN,NaN,NaT,NaN,0.94,0.925,0.875,NaN,0.083894,0.082506,0.077892
2974,2023-01-10,139.55,NaN,NaN,NaT,NaN,0.94,0.925,0.875,NaN,0.083894,0.082506,0.077892
2975,2023-01-11,139.97,NaN,NaN,NaT,NaN,0.94,0.925,0.875,NaN,0.083633,0.082250,0.077650
2976,2023-01-12,139.40,NaN,NaN,NaT,NaN,0.94,0.925,0.875,NaN,0.083988,0.082598,0.077978


In [20]:
# Plotar o DY em 3, 6 e 12 meses
fig = go.Figure()
fig.add_trace(go.Scatter(x=knri['datneg'], y=knri['dy_3m'] * 100, name='DY 3M'))
fig.add_trace(go.Scatter(x=knri['datneg'], y=knri['dy_6m'] * 100, name='DY 6M'))
fig.add_trace(go.Scatter(x=knri['datneg'], y=knri['dy_12m'] * 100, name='DY 12M'))
fig.update_layout(
    title="Dividend Yield (DY) do KNRI11",
    xaxis_title="Data",
    yaxis_title="DY (%)",
    title_x=0.5,
    title_y=0.85,
)
fig.show()


In [47]:
# Importar IPCA+
# https://www.tesourotransparente.gov.br/ckan/dataset/df56aa42-484a-4a59-8184-7676580c81e3/resource/796d2059-14e9-44e3-80c9-2d9e30b405c1/download/PrecoTaxaTesouroDireto.csv
df_tn = (pd
    .read_csv("data/PrecoTaxaTesouroDireto.csv", sep=";", decimal=",", thousands=".", parse_dates=["Data Base", "Data Vencimento"], dayfirst=True)
    # TaxaCompraManha	PUCompraManha	PUVendaManha	PUBaseManha
    .drop(columns=["Taxa Compra Manha", "PU Compra Manha", "PU Venda Manha", "PU Base Manha"])
    .sort_values(by="Data Base", ignore_index=True)
)
df_tn

,Tipo Titulo,Data Vencimento,Data Base,Taxa Venda Manha
0,Tesouro IGPM+ com Juros Semestrais,2031-01-01,2004-12-31,8.34
1,Tesouro Prefixado,2006-01-01,2004-12-31,18.21
2,Tesouro Prefixado,2006-07-01,2004-12-31,17.75
3,Tesouro IPCA+ com Juros Semestrais,2006-08-15,2004-12-31,8.73
4,Tesouro IPCA+ com Juros Semestrais,2009-05-15,2004-12-31,8.82
...,...,...,...,...
127246,Tesouro IPCA+,2029-05-15,2023-01-18,6.05
127247,Tesouro Prefixado com Juros Semestrais,2031-01-01,2023-01-18,12.60
127248,Tesouro Prefixado com Juros Semestrais,2025-01-01,2023-01-18,12.55
127249,Tesouro IPCA+ com Juros Semestrais,2030-08-15,2023-01-18,6.04


In [48]:
# remove space from column names
for col in df_tn.columns:
    df_tn.rename(columns={col: col.replace(" ", "")}, inplace=True)
df_tn["TempoVencimento"] = df_tn["DataVencimento"] - df_tn["DataBase"]
df_tn["TempoVencimento"] = df_tn["TempoVencimento"].dt.days / 365
df_tn

,TipoTitulo,DataVencimento,DataBase,TaxaVendaManha,TempoVencimento
0,Tesouro IGPM+ com Juros Semestrais,2031-01-01,2004-12-31,8.34,26.019178
1,Tesouro Prefixado,2006-01-01,2004-12-31,18.21,1.002740
2,Tesouro Prefixado,2006-07-01,2004-12-31,17.75,1.498630
3,Tesouro IPCA+ com Juros Semestrais,2006-08-15,2004-12-31,8.73,1.621918
4,Tesouro IPCA+ com Juros Semestrais,2009-05-15,2004-12-31,8.82,4.372603
...,...,...,...,...,...
127246,Tesouro IPCA+,2029-05-15,2023-01-18,6.05,6.326027
127247,Tesouro Prefixado com Juros Semestrais,2031-01-01,2023-01-18,12.60,7.958904
127248,Tesouro Prefixado com Juros Semestrais,2025-01-01,2023-01-18,12.55,1.956164
127249,Tesouro IPCA+ com Juros Semestrais,2030-08-15,2023-01-18,6.04,7.578082


In [49]:
df_tn = (df_tn
    .query('TipoTitulo == "Tesouro IPCA+ com Juros Semestrais"')
    .query('TempoVencimento > 10')
    .sort_values(by=["DataBase", "TempoVencimento"], ignore_index=True)
    .drop_duplicates(subset="DataBase", keep="first")
    .rename(columns={"DataBase": "datneg", "TaxaVendaManha": "TesouroIPCA+"})
)
df_tn

,TipoTitulo,DataVencimento,datneg,TesouroIPCA+,TempoVencimento
0,Tesouro IPCA+ com Juros Semestrais,2015-05-15,2004-12-31,8.84,10.375342
3,Tesouro IPCA+ com Juros Semestrais,2015-05-15,2005-01-03,8.84,10.367123
6,Tesouro IPCA+ com Juros Semestrais,2015-05-15,2005-01-04,8.84,10.364384
9,Tesouro IPCA+ com Juros Semestrais,2015-05-15,2005-01-05,8.84,10.361644
12,Tesouro IPCA+ com Juros Semestrais,2015-05-15,2005-01-06,8.84,10.358904
...,...,...,...,...,...
16040,Tesouro IPCA+ com Juros Semestrais,2035-05-15,2023-01-12,6.12,12.345205
16045,Tesouro IPCA+ com Juros Semestrais,2035-05-15,2023-01-13,6.19,12.342466
16050,Tesouro IPCA+ com Juros Semestrais,2035-05-15,2023-01-16,6.19,12.334247
16055,Tesouro IPCA+ com Juros Semestrais,2035-05-15,2023-01-17,6.24,12.331507


In [50]:
# Unir os dois dataframes
knri = knri.merge(df_tn).sort_values("datneg")
# Remover as colunas com as médias móveis
knri.drop(columns=["div_mms3", "div_mms6", "div_mms12"], inplace=True)
knri

,datneg,preult,dividendo,ano_mes,mes,indice,div_mms6_real,dy_3m,dy_6m,dy_12m,TipoTitulo,DataVencimento,TesouroIPCA+,TempoVencimento
0,2011-01-04,100.00,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,Tesouro IPCA+ com Juros Semestrais,2024-08-15,5.87,13.621918
1,2011-01-05,99.70,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,Tesouro IPCA+ com Juros Semestrais,2024-08-15,5.90,13.619178
2,2011-01-06,99.70,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,Tesouro IPCA+ com Juros Semestrais,2024-08-15,5.91,13.616438
3,2011-01-07,99.50,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,Tesouro IPCA+ com Juros Semestrais,2024-08-15,5.94,13.613699
4,2011-01-10,95.50,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,Tesouro IPCA+ com Juros Semestrais,2024-08-15,6.00,13.605479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2967,2023-01-09,139.55,NaN,NaN,NaT,NaN,NaN,0.083894,0.082506,0.077892,Tesouro IPCA+ com Juros Semestrais,2035-05-15,6.42,12.353425
2968,2023-01-10,139.55,NaN,NaN,NaT,NaN,NaN,0.083894,0.082506,0.077892,Tesouro IPCA+ com Juros Semestrais,2035-05-15,6.35,12.350685
2969,2023-01-11,139.97,NaN,NaN,NaT,NaN,NaN,0.083633,0.082250,0.077650,Tesouro IPCA+ com Juros Semestrais,2035-05-15,6.30,12.347945
2970,2023-01-12,139.40,NaN,NaN,NaT,NaN,NaN,0.083988,0.082598,0.077978,Tesouro IPCA+ com Juros Semestrais,2035-05-15,6.12,12.345205


In [51]:
# Calcular o spread
knri["spread"] = knri["dy_12m"] - knri["TesouroIPCA+"]
knri

,datneg,preult,dividendo,ano_mes,mes,indice,div_mms6_real,dy_3m,dy_6m,dy_12m,TipoTitulo,DataVencimento,TesouroIPCA+,TempoVencimento,spread
0,2011-01-04,100.00,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,Tesouro IPCA+ com Juros Semestrais,2024-08-15,5.87,13.621918,NaN
1,2011-01-05,99.70,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,Tesouro IPCA+ com Juros Semestrais,2024-08-15,5.90,13.619178,NaN
2,2011-01-06,99.70,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,Tesouro IPCA+ com Juros Semestrais,2024-08-15,5.91,13.616438,NaN
3,2011-01-07,99.50,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,Tesouro IPCA+ com Juros Semestrais,2024-08-15,5.94,13.613699,NaN
4,2011-01-10,95.50,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,Tesouro IPCA+ com Juros Semestrais,2024-08-15,6.00,13.605479,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2967,2023-01-09,139.55,NaN,NaN,NaT,NaN,NaN,0.083894,0.082506,0.077892,Tesouro IPCA+ com Juros Semestrais,2035-05-15,6.42,12.353425,-6.342108
2968,2023-01-10,139.55,NaN,NaN,NaT,NaN,NaN,0.083894,0.082506,0.077892,Tesouro IPCA+ com Juros Semestrais,2035-05-15,6.35,12.350685,-6.272108
2969,2023-01-11,139.97,NaN,NaN,NaT,NaN,NaN,0.083633,0.082250,0.077650,Tesouro IPCA+ com Juros Semestrais,2035-05-15,6.30,12.347945,-6.222350
2970,2023-01-12,139.40,NaN,NaN,NaT,NaN,NaN,0.083988,0.082598,0.077978,Tesouro IPCA+ com Juros Semestrais,2035-05-15,6.12,12.345205,-6.042022


In [58]:
# Fazer o mesmo gráfico, mas com o Plotly
# Plotar o DY em 3, 6 e 12 meses
fig = go.Figure()
fig.add_trace(go.Scatter(x=knri['datneg'], y=knri['dy_6m'] * 100, name='KNRI DY 6M'))
fig.add_trace(go.Scatter(x=knri['datneg'], y=knri['TesouroIPCA+'], name='TesouroIPCA+'))
fig.update_layout(
    title="KNRI11 dividend yield vs TesouroIPCA+",
    xaxis_title="Data",
    yaxis_title="DY (%)",
    title_x=0.5,
    title_y=0.85,
)
fig.update_layout(
    title={
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    font_family="Open Sans",
    # legend_title_text='Variáveis',
    # template="plotly_dark",
    legend=dict(
        x=0.01,
        y=0.99,
        traceorder="normal",
    ),
)
fig.update_traces(
    selector=dict(name="dy_12m"),
    name="Dividend Yield (12m)"
)
# Inserir texto "" na parte inferior do gráfico
fig.add_annotation(
    x=0.5,
    y=-0.3,
    xref="paper",
    yref="paper",
    text="Dividend Yield (média de 6 meses anualizada) e TesouroIPCA+ (títulos com venc. > 10 anos)",
    showarrow=False,
    font=dict(
        family="Open Sans",
        size=12,
        color="black"
    )
)


fig.show()

In [17]:
tr.loc[15:30]

,datneg,preult,div_data_ex,div_data_pag,fr_p1,fr_p2,fr_p3
15,2011-01-27,97.00,NaN,NaN,NaN,NaN,NaN
16,2011-01-28,96.60,NaN,NaN,NaN,NaN,NaN
17,2011-01-31,96.99,NaN,NaN,NaN,NaN,NaN
18,2011-02-01,96.99,0.675,NaN,1.006959,NaN,1.007008
19,2011-02-02,99.00,NaN,NaN,NaN,NaN,NaN
20,2011-02-03,98.50,NaN,NaN,NaN,NaN,NaN
21,2011-02-04,99.80,NaN,NaN,NaN,NaN,NaN
22,2011-02-07,97.00,NaN,NaN,NaN,NaN,NaN
23,2011-02-08,96.50,NaN,NaN,NaN,NaN,NaN
24,2011-02-09,97.00,NaN,NaN,NaN,NaN,NaN


In [18]:
# Preencher os valores NaN com 1 em fr_p1, fr_p2 e fr_p3 para calcular o produtório
tr[["fr_p1", "fr_p2", "fr_p3"]] = tr[["fr_p1", "fr_p2", "fr_p3"]].fillna(1)
tr

,datneg,preult,div_data_ex,div_data_pag,fr_p1,fr_p2,fr_p3
0,2011-01-04,100.00,NaN,NaN,1.0,1.000000,1.0
1,2011-01-05,99.70,NaN,NaN,1.0,1.000000,1.0
2,2011-01-06,99.70,NaN,NaN,1.0,1.000000,1.0
3,2011-01-07,99.50,NaN,NaN,1.0,1.000000,1.0
4,2011-01-10,95.50,NaN,NaN,1.0,1.000000,1.0
...,...,...,...,...,...,...,...
2973,2023-01-09,139.55,NaN,NaN,1.0,1.000000,1.0
2974,2023-01-10,139.55,NaN,NaN,1.0,1.000000,1.0
2975,2023-01-11,139.97,NaN,NaN,1.0,1.000000,1.0
2976,2023-01-12,139.40,NaN,NaN,1.0,1.000000,1.0


In [19]:
"""
Calcular a participação acumulada
Políticas de reinvestimento de dividendos:
  p1 = reinvestimento pelo preço de fechamento da data-ex
  p2 = reinvestimento pelo preço de fechamento da data-pag
  p3 = reinvestimento pelo preço de abertura teórico da data-ex
Participação acumulada = produtório dos fatores de reinvestimento
"""
tr["p1"] = tr["fr_p1"].cumprod()
tr["p2"] = tr["fr_p2"].cumprod()
tr["p3"] = tr["fr_p3"].cumprod()
# As colunas "fr_p1", "fr_p2" e "fr_p3" não serão mais usadas
cols_remove = ["fr_p1", "fr_p2", "fr_p3","div_data_ex", "div_data_pag"]
tr.drop(columns=cols_remove, inplace=True)
tr

,datneg,preult,p1,p2,p3
0,2011-01-04,100.00,1.00000,1.000000,1.000000
1,2011-01-05,99.70,1.00000,1.000000,1.000000
2,2011-01-06,99.70,1.00000,1.000000,1.000000
3,2011-01-07,99.50,1.00000,1.000000,1.000000
4,2011-01-10,95.50,1.00000,1.000000,1.000000
...,...,...,...,...,...
2973,2023-01-09,139.55,2.32401,2.307507,2.324269
2974,2023-01-10,139.55,2.32401,2.307507,2.324269
2975,2023-01-11,139.97,2.32401,2.307507,2.324269
2976,2023-01-12,139.40,2.32401,2.307507,2.324269


In [20]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=tr["datneg"],
        y=tr["p1"],
        mode="lines",
        name="quantidades de cotas",
    )
)
fig.update_layout(
    font=dict(family="Fira Code", size=11, color="black"),
    title="Participação Acumulada no KNRI11",
    title_x=0.5,
    title_y=0.85,
    xaxis_title="Data",
    yaxis_title="Participação Acumulada",
)
fig.show()

In [21]:
print(tr.tail().to_markdown( tablefmt="psql", index=False, numalign="right", floatfmt=".4f"))

+---------------------+----------+--------+--------+--------+
| datneg              |   preult |     p1 |     p2 |     p3 |
|---------------------+----------+--------+--------+--------|
| 2023-01-09 00:00:00 | 139.5500 | 2.3240 | 2.3075 | 2.3243 |
| 2023-01-10 00:00:00 | 139.5500 | 2.3240 | 2.3075 | 2.3243 |
| 2023-01-11 00:00:00 | 139.9700 | 2.3240 | 2.3075 | 2.3243 |
| 2023-01-12 00:00:00 | 139.4000 | 2.3240 | 2.3075 | 2.3243 |
| 2023-01-13 00:00:00 | 138.8000 | 2.3240 | 2.3241 | 2.3243 |
+---------------------+----------+--------+--------+--------+


In [22]:
tr["preult_tr"] = tr["preult"] * tr["p1"]
tr

,datneg,preult,p1,p2,p3,preult_tr
0,2011-01-04,100.00,1.00000,1.000000,1.000000,100.000000
1,2011-01-05,99.70,1.00000,1.000000,1.000000,99.700000
2,2011-01-06,99.70,1.00000,1.000000,1.000000,99.700000
3,2011-01-07,99.50,1.00000,1.000000,1.000000,99.500000
4,2011-01-10,95.50,1.00000,1.000000,1.000000,95.500000
...,...,...,...,...,...,...
2973,2023-01-09,139.55,2.32401,2.307507,2.324269,324.315603
2974,2023-01-10,139.55,2.32401,2.307507,2.324269,324.315603
2975,2023-01-11,139.97,2.32401,2.307507,2.324269,325.291687
2976,2023-01-12,139.40,2.32401,2.307507,2.324269,323.967001


In [23]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=tr["datneg"],
        y=tr["preult"],
        mode="lines",
        name="Cotação de Mercado",
        line=dict(color="#e86f00"),
    )
)
fig.add_trace(
    go.Scatter(
        x=tr["datneg"],
        y=tr["preult_tr"],
        mode="lines",
        name="Cotação Total Return",
        line=dict(color="#02878e"),
    )
)
fig.update_layout(
    font=dict(family="Fira Code", size=11, color="black"),
    title="KNRI11<br>Cotação de Mercado vs Cotação Total Return",
    title_x=0.5,
    title_y=0.85,
    xaxis_title="Data",
    yaxis_title="Valor (R$)",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)
fig.show()

In [24]:
# Retorno nominal do fundo
retorno_nominal_periodo = (tr['preult_tr'].iloc[-1] - tr['preult_tr'].iloc[0]) / tr['preult_tr'].iloc[0]
print(f'Retorno total no período  = {retorno_nominal_periodo:.2%}')
years = (tr['datneg'].iloc[-1] - tr['datneg'].iloc[-0]).days / 365
print(f'Total de anos no periodo = {years:.2f} anos')
# Retorno nominal  anualizado
retorno_nominal_periodo_anualizado = (retorno_nominal_periodo + 1) ** (1 / years) - 1
print(f'Retorno nominal anualizado = {retorno_nominal_periodo_anualizado:.2%}')

Retorno total no período  = 222.57%
Total de anos no periodo = 12.03 anos
Retorno nominal anualizado = 10.22%


In [27]:
# Salvar preult_tr em um arquivo csv
tr[["datneg", "preult_tr"]].to_csv("./data/knri_tr.csv", index=False)